In [137]:
import pandas as pd
from fuzzywuzzy import fuzz

# DataFrames A, B, and C
A = pd.DataFrame({'id': ['1', '2', '3', '4', '5', '6'],
                  'name': ['robert', 'mariana', 'jhon', 'bogdan', 'alex', '9'],
                  'occupation': ['constructor', 'witch', 'buldozer', 'pirate', 'doctor', '5']})

B = pd.DataFrame({'id': ['1', '2', '3', '4', '5', '6'],
                  'name': ['dragos', 'muntean', 'palavra', 'javra', 'caine', '10'],
                  'age': ['19', '22', '66', '23', '55', '']})

C = pd.DataFrame({'id': ['1', '2', '3', '4', '5', '6'],
                  'name': ['robert', 'munte', 'pala', 'juva', 'catel', '19'],
                  'timeLeft': ['30', '28', '36', '5', '100', '12']})

# Merge A and B
AB = pd.merge(A, B, how="inner", on="id")
for index, record in AB.iterrows():
    AB.at[index, "name"] = str(AB.at[index, "name_x"]) + "," + str(AB.at[index, "name_y"])
AB.drop(columns=["name_x", "name_y"], inplace=True)

# Merge AB with C
ABC = pd.merge(AB, C, how='inner', on="id")
for index, record in ABC.iterrows():
    ABC.at[index, "name"] = str(ABC.at[index, "name_x"]) + "," + str(ABC.at[index, "name_y"])
ABC.drop(columns=["name_y","name_x"], inplace=True)

print(ABC)
# Process to compute similarity scores and pick the name with the highest score
for index, record in ABC.iterrows():
    names = ABC.at[index, "name"].split(',')  # Split the concatenated names
    scores = []
    
    # Calculate similarity scores for each name against the others
    for i in range(len(names)):
        score = 0
        for j in range(len(names)):
            if i != j:
                score += fuzz.ratio(names[i], names[j])
        scores.append(score)

    # Find the name with the highest score and assign it back to ABC['name']
    max_score_index = scores.index(max(scores))
    ABC.at[index, "name"] = names[max_score_index]

# Print the final ABC DataFrame
print(ABC)


  id   occupation age timeLeft                   name
0  1  constructor  19       30   robert,dragos,robert
1  2        witch  22       28  mariana,muntean,munte
2  3     buldozer  66       36      jhon,palavra,pala
3  4       pirate  23        5      bogdan,javra,juva
4  5       doctor  55      100       alex,caine,catel
5  6            5           12                9,10,19
  id   occupation age timeLeft     name
0  1  constructor  19       30   robert
1  2        witch  22       28  muntean
2  3     buldozer  66       36  palavra
3  4       pirate  23        5     juva
4  5       doctor  55      100    caine
5  6            5           12       19
